In [7]:
from sklearn import datasets
import numpy as np
import pandas as pdf
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
import sympy as sym
from sklearn.cluster import SpectralClustering, KMeans
from scipy.spatial import distance
import sympy as sym
from scipy.sparse import csr_matrix 
k=4
iris = datasets.load_iris()
X = iris.data  
y = iris.target

In [8]:
import time 
t_begin = time.time()
A= pairwise_distances(X, metric="euclidean")
DNN, NN = np.sort(A), np.argsort(A)
NN = NN[:,1:k+1]
DNN = DNN[:,1:k+1]
W=csr_matrix((X.shape[0], X.shape[0])).toarray()
#W = np.zeros((X.shape[0], X.shape[0]))
for i in range(W.shape[0]):
    for j in range(W.shape[0]):
        if (j in NN[i])or(i in NN[j]):
            W[i,j] = np.exp(-(A[i,j]**2)/1000000)
        else:
            W[i][j]=0
D = np.diag(np.sum(np.array(W), axis=1))
L = D - W
e1, v = np.linalg.eig(L)
idx=np.argsort(e1)
e1=e1[idx]
eps=np.max(e1)
L=L/eps
XTX=np.dot(X,X.T)
e2,v2 = np.linalg.eig(XTX)
idx2=np.argsort(e2)
e2=e2[idx2]
lam=np.max(e2)
XTX=XTX/lam
B=0.5
GB=(1-B)*(np.identity(X.shape[0])-XTX)+(B*L)
e3, v3 = np.linalg.eig(GB)
idx3=np.argsort(e3)
e3=e3[idx3]
U = np.array(v3.real[:,idx3[0:3]])
km = KMeans(init='k-means++', n_clusters=3)
km.fit(U)
t_end = time.time()
t_end-t_begin

0.20248889923095703

In [9]:
from sklearn.metrics.cluster import normalized_mutual_info_score
normalized_mutual_info_score(y,km.labels_,average_method='arithmetic')

0.8850620966553382

In [10]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(y,km.labels_)

0.9037675791580496